In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

ModuleNotFoundError: No module named 'azureml.core'

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

amlcompute_cluster_name='cpu_cluster'

compute_config=AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)

aml_compute=ComputeTarget.create(ws,amlcompute_cluster_name,compute_config)

aml_compute.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling(
     { '--batch-size': choice(20,50,100),
       '--first-layer-neurons': choice(10,50,200,300,500),
       '--second-layer-neurons': choice(10,50,200,500),
       '--learning-rate': loguniform(-6,-1)
     } 
    
) 

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = TensorFlow(source_directory=script_folder,
                 script_params= {'--data-folder':dataset.as_named_input('bank_marketing').as_mount()},
                 compute_target=compute_target,
                 entry_script='train.py',
                 framework_version='2.0',
                 pip_packages=['keras<=2.3.1','azureml-dataset-rutime[pandas,fuse]']
                )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr=exp.submit(config=hyperdrive_config)
RunDetails(hdr).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run=run.get_best_run_by_primary_metric()
best_run.get_details()

#best_model=LogisticRegression().fit(x_train,y_train)
#joblib.dump(best_model,'best_hdrvive_model.pkl')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path ='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
dataset = Dataset.Tabular.from_delimited_files(path=web_path)
ds = dataset.to_pandas_dataframe()

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classifaction',
    primary_metric='Accuracy',
    training_data=dataset,
    label_column_name=,
    n_cross_validations=2)

In [2]:
# Submit your automl run

automl_run=exp.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

best,fitted = automl_run.get_output()
print(best)

joblib.dump(fitted,'best_automl_model.pkl')